### Notas

Objetivo de reduzir custo de manutenção </br>
prever falhas em equipamentos 20 ciclos à frente (classificação) </br>
prever quantos ciclos o equipamento terá (regressão) </br>
mudar planos de manutenção de acordo com as predições de falhas </br>
A frequência de manutenção acontecem em um período fixo


## Limpeza de dados

In [1]:
import pandas as pd
import numpy as np
import pickle

from classes.dataprep import DataPrep

### Descrição dos dados

Asset id -> id da máquina </br>
runtime -> número de ciclos que a máquina teve até falhar </br>
engine1-3 -> configurações operacionais da máquina </br>
t1-t21 -> dados de sensores </br>

In [2]:
col_names = ['asset_id','runtime','engine1','engine2','engine3','t1','t2','t3','t4','t5',
             't6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18',
             't19','t20','t21']

### Dados de treino

In [3]:
df_train_raw = pd.read_csv('data/PM_train.txt', sep=" ", header= None)
df_train_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN


#### As colunas 26 e 27 estão nulas, estão serão removidas

In [4]:
df_train_raw.isna().sum()

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26    20631
27    20631
dtype: int64

In [5]:
df_train_raw.drop([26, 27], axis = 1, inplace = True)

In [6]:
# Renomeando colunas

df_train_raw.columns = col_names
df_train_raw.head()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


### Olhando o resumo dos dados, temos alguns insights

* Cada asset id representa uma máquina, olhando o mínimo e máximo dessa coluna <b>existem 100 motores diferentes<b>
* As máquinas tem entre <b>1 ciclo até 362<b> tendo uma média de <b>108 ciclos<b>
* o último ciclo de cada máquina representa quando que ele falhou

In [7]:
df_train_raw.describe()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.0,20631.00,20631.000000,20631.000000,20631.000000,2.063100e+04,...,20631.000000,20631.000000,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000
mean,51.506568,108.807862,-0.000009,0.000002,100.0,518.67,642.680934,1590.523119,1408.933782,1.462000e+01,...,521.413470,2388.096152,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705
std,29.227633,68.880990,0.002187,0.000293,0.0,0.00,0.500053,6.131150,9.000605,1.776400e-15,...,0.737553,0.071919,19.076176,0.037505,1.387812e-17,1.548763,0.0,0.0,0.180746,0.108251
min,1.000000,1.000000,-0.008700,-0.000600,100.0,518.67,641.210000,1571.040000,1382.250000,1.462000e+01,...,518.690000,2387.880000,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200
25%,26.000000,52.000000,-0.001500,-0.000200,100.0,518.67,642.325000,1586.260000,1402.360000,1.462000e+01,...,520.960000,2388.040000,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800
50%,52.000000,104.000000,0.000000,0.000000,100.0,518.67,642.640000,1590.100000,1408.040000,1.462000e+01,...,521.480000,2388.090000,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900
75%,77.000000,156.000000,0.001500,0.000300,100.0,518.67,643.000000,1594.380000,1414.555000,1.462000e+01,...,521.950000,2388.140000,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800
max,100.000000,362.000000,0.008700,0.000600,100.0,518.67,644.530000,1616.910000,1441.490000,1.462000e+01,...,523.380000,2388.560000,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400


In [8]:
# nenhum dado nulo
df_train_raw.isnull().sum()

asset_id    0
runtime     0
engine1     0
engine2     0
engine3     0
t1          0
t2          0
t3          0
t4          0
t5          0
t6          0
t7          0
t8          0
t9          0
t10         0
t11         0
t12         0
t13         0
t14         0
t15         0
t16         0
t17         0
t18         0
t19         0
t20         0
t21         0
dtype: int64

### Dados de teste

In [9]:
df_test_raw = pd.read_csv('data/PM_test.txt', sep=" ", header= None)
df_test_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,NaN,NaN
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,NaN,NaN
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,NaN,NaN
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,NaN,NaN
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,NaN,NaN


In [10]:
df_test_raw.drop([26, 27], axis = 1, inplace = True)

In [11]:
df_test_raw.describe()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
count,13096.000000,13096.000000,13096.000000,13096.000000,13096.0,13096.00,13096.000000,13096.000000,13096.000000,1.309600e+04,...,13096.000000,13096.000000,13096.000000,13096.000000,1.309600e+04,13096.000000,13096.0,13096.0,13096.000000,13096.000000
mean,51.543907,76.836515,-0.000011,0.000004,100.0,518.67,642.475088,1588.099204,1404.735362,1.462000e+01,...,521.747724,2388.071025,8138.947817,8.425844,3.000000e-02,392.571625,2388.0,100.0,38.892502,23.335743
std,28.289423,53.057749,0.002203,0.000294,0.0,0.00,0.400899,5.003274,6.688309,1.776425e-15,...,0.559627,0.056934,10.188605,0.029009,1.040874e-17,1.233577,0.0,0.0,0.141681,0.084120
min,1.000000,1.000000,-0.008200,-0.000600,100.0,518.67,641.130000,1569.040000,1384.390000,1.462000e+01,...,519.380000,2387.890000,8108.500000,8.332800,3.000000e-02,389.000000,2388.0,100.0,38.310000,22.935400
25%,28.000000,33.000000,-0.001500,-0.000200,100.0,518.67,642.197500,1584.600000,1399.950000,1.462000e+01,...,521.380000,2388.030000,8132.310000,8.405600,3.000000e-02,392.000000,2388.0,100.0,38.800000,23.281600
50%,52.000000,69.000000,0.000000,0.000000,100.0,518.67,642.460000,1587.990000,1404.440000,1.462000e+01,...,521.780000,2388.070000,8138.390000,8.424900,3.000000e-02,393.000000,2388.0,100.0,38.900000,23.339200
75%,76.000000,113.000000,0.001500,0.000300,100.0,518.67,642.740000,1591.362500,1409.050000,1.462000e+01,...,522.150000,2388.110000,8144.360000,8.444300,3.000000e-02,393.000000,2388.0,100.0,38.990000,23.393600
max,100.000000,303.000000,0.007800,0.000700,100.0,518.67,644.300000,1607.550000,1433.360000,1.462000e+01,...,523.760000,2388.320000,8220.480000,8.541400,3.000000e-02,397.000000,2388.0,100.0,39.410000,23.641900


In [12]:
# Renomeando colunas

df_test_raw.columns = col_names
df_test_raw.head()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


### Suavização de dados e preparação para treino de modelo

* Para um melhor desempenho no modelo, foi feita uma suavização dos dados de sensor <br>
    duas técnicas foram utilizadas, [média móvel](https://en.wikipedia.org/wiki/Moving_average) e também [desvio padrão móvel](https://doc-archives.microstrategy.com/producthelp/10.10/FunctionsRef/Content/FuncRef/MovingStDev__moving_standard_deviation_.htm) <br>

* O cálculo do RUL foi feito da seguinte forma: <b>último ciclo de um asset_id - runtime</b>, e será usado para regressão<br>
    para a classificação binária foi atribuído <b>0 caso a máquina não esteja nos seus últimos 20 ciclos e 1 para o oposto</b> 
    
* Para isso foi criada uma classe em ./classes/dataprep.py para algumas funções de preparação de dados <br>


### Dados de treino

In [13]:
data_prep_train = DataPrep(df_train_raw)

In [14]:
# dados de sensor suavzados
df_train_smooth = data_prep_train.smooth_data(5)
df_train_smooth.head()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,sd12,sd13,sd14,sd15,sd16,sd17,sd18,sd19,sd20,sd21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,0.438406,0.035355,5.041671,0.008697,0.0,0.000000,0.0,0.0,0.042426,0.003253
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,0.404475,0.026458,3.717450,0.007640,0.0,1.154701,0.0,0.0,0.055076,0.044573
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,0.495950,0.029439,3.050906,0.028117,0.0,1.000000,0.0,0.0,0.076322,0.037977
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,0.432574,0.025884,2.651326,0.025953,0.0,1.095445,0.0,0.0,0.073621,0.033498


#### Aqui é aplicado o cálculo do RUL e a coluna label_failure, que caso seja 1 significa que a máquina está a 20 ou menos ciclos de falhar

In [15]:
df_train = data_prep_train.add_labels(df_train_smooth, 20, True)
df_train.head()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,sd14,sd15,sd16,sd17,sd18,sd19,sd20,sd21,rul,failure_label
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,191,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,5.041671,0.008697,0.0,0.000000,0.0,0.0,0.042426,0.003253,190,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,3.717450,0.007640,0.0,1.154701,0.0,0.0,0.055076,0.044573,189,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,3.050906,0.028117,0.0,1.000000,0.0,0.0,0.076322,0.037977,188,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2.651326,0.025953,0.0,1.095445,0.0,0.0,0.073621,0.033498,187,0


In [16]:
# salva objeto
with open('data/df_train.pkl', 'wb') as p:
    pickle.dump(df_train, p)

### Dados de Teste

In [17]:
data_prep_teste = DataPrep(df_test_raw)

In [18]:
# dados de sensor suavzados
df_test_smooth = data_prep_teste.smooth_data(5)
df_test_smooth.head()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,sd12,sd13,sd14,sd15,sd16,sd17,sd18,sd19,sd20,sd21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,0.311127,0.021213,9.948992,0.017607,0.0,0.707107,0.0,0.0,0.113137,0.012799
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,0.220681,0.017321,7.179807,0.032155,0.0,0.577350,0.0,0.0,0.113725,0.021642
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,0.337182,0.015000,5.890096,0.027781,0.0,0.957427,0.0,0.0,0.093095,0.020353
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,0.329742,0.014142,5.222300,0.024080,0.0,1.303840,0.0,0.0,0.080623,0.020672


In [19]:
df_test = data_prep_teste.add_labels(df_test_smooth, 20, False)
df_test.head()

,asset_id,runtime,engine1,engine2,engine3,t1,t2,t3,t4,t5,...,sd12,sd13,sd14,sd15,sd16,sd17,sd18,sd19,sd20,sd21
30,1,31,-0.0006,0.0004,100.0,518.67,642.58,1581.22,1398.91,14.62,...,0.261591,0.028810,3.302941,0.021174,0.0,0.707107,0.0,0.0,0.124016,0.025037
79,2,49,0.0018,-0.0001,100.0,518.67,642.55,1586.59,1410.83,14.62,...,0.450255,0.008367,1.370062,0.011429,0.0,1.095445,0.0,0.0,0.069785,0.027820
205,3,126,-0.0016,0.0004,100.0,518.67,642.88,1589.75,1418.89,14.62,...,0.418234,0.029665,2.979090,0.030924,0.0,0.836660,0.0,0.0,0.143631,0.018966
311,4,106,0.0012,0.0004,100.0,518.67,642.78,1594.53,1406.88,14.62,...,0.363291,0.027019,0.622953,0.021249,0.0,1.341641,0.0,0.0,0.126372,0.021106
409,5,98,-0.0013,-0.0004,100.0,518.67,642.27,1589.94,1419.36,14.62,...,0.260231,0.023022,1.923609,0.009627,0.0,0.547723,0.0,0.0,0.091924,0.101100


In [22]:
# salva objeto
with open('data/df_test.pkl', 'wb') as p:
    pickle.dump(df_test, p)